# Prediction model with Tensorflow

In [1]:
%pylab inline
import tensorflow as tf
import sys

# Include additional module
include_path = '../include'
if include_path not in sys.path:
    sys.path.append(include_path)
from tensorflow_oop import *

Populating the interactive namespace from numpy and matplotlib


## Create dataset

### Load dump of dataset

In [2]:
DUMP_PATH = '../data/moscow.dump'

print 'Loading dataset...'
dataset = TFDataset()
dataset.load(DUMP_PATH)
print 'Dataset shape:', dataset.size_, dataset.data_shape_, '->', dataset.labels_shape_

Loading dataset...
Dataset shape: 36566 [32, 3] -> [1, 3]


### Set batch size

In [3]:
BATCH_SIZE = 32

dataset.set_batch_size(BATCH_SIZE)

### Split dataset

In [4]:
TRAIN_RATE = 0.70
VAL_RATE = 0.15
TEST_RATE = 0.15

print 'Splitting...'
train_set, val_set, test_set = dataset.split(TRAIN_RATE, VAL_RATE, TEST_RATE, shuffle=False)
print 'Traininig  set shape:', train_set.size_, train_set.data_shape_, '->', train_set.labels_shape_
print 'Validation set shape:', val_set.size_, val_set.data_shape_, '->', val_set.labels_shape_
print 'Testing    set shape:', test_set.size_, test_set.data_shape_, '->', test_set.labels_shape_

Splitting...
Traininig  set shape: 25596 [32, 3] -> [1, 3]
Validation set shape: 5486 [32, 3] -> [1, 3]
Testing    set shape: 5484 [32, 3] -> [1, 3]


### Shuffle datasets

In [5]:
train_set.shuffle()
val_set.shuffle()
test_set.shuffle()

## Define model

In [6]:
class TFWeatherForecast(TFRegressor):
    def inference(self, inputs, kwargs={}):
        hidden_size = 100
        cell = tf.nn.rnn_cell.LSTMCell(hidden_size, state_is_tuple=True)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell,
                                                     inputs,
                                                     dtype=tf.float32,
                                                     time_major=False)
        W = tf.Variable(tf.truncated_normal([hidden_size, self.outputs_shape_[-1]]))
        b = tf.Variable(tf.zeros([self.outputs_shape_[-1]]))
        outputs = tf.nn.xw_plus_b(rnn_outputs[:,-1], W, b)
        return tf.reshape(outputs, [-1] + self.outputs_shape_)

In [7]:
LOG_DIR = '/tmp/prediction-model'
INPUTS_SHAPE = dataset.data_shape_
OUTPUTS_SHAPE = dataset.labels_shape_

model = TFWeatherForecast(log_dir=LOG_DIR, inputs_shape=INPUTS_SHAPE, outputs_shape=OUTPUTS_SHAPE)

Start initializing model...
Finish initializing model.


## Train model

In [9]:
EPOCH_COUNT = 23

model.fit(train_set, EPOCH_COUNT, val_set=val_set)

Start training iteration...
Epoch 1/23:   loss = 0.109162   (61.495 sec)
Epoch 2/23:   loss = 0.105373   (61.653 sec)
Epoch 3/23:   loss = 0.098353   (61.931 sec)
Epoch 4/23:   loss = 0.093958   (62.268 sec)
Epoch 5/23:   loss = 0.088209   (61.602 sec)
Epoch 6/23:   loss = 0.087645   (62.137 sec)
Epoch 7/23:   loss = 0.086160   (61.596 sec)
Epoch 8/23:   loss = 0.086382   (61.667 sec)
Epoch 9/23:   loss = 0.086553   (61.468 sec)
Epoch 10/23:   loss = 0.085446   (61.982 sec)
Epoch 11/23:   loss = 0.088529   (64.612 sec)
Epoch 12/23:   loss = 0.085449   (61.904 sec)
Epoch 13/23:   loss = 0.085235   (61.778 sec)
Epoch 14/23:   loss = 0.087202   (62.191 sec)
Epoch 15/23:   loss = 0.087553   (61.337 sec)
Epoch 16/23:   loss = 0.087610   (61.453 sec)
Epoch 17/23:   loss = 0.088355   (62.157 sec)
Epoch 18/23:   loss = 0.092566   (61.314 sec)
Epoch 19/23:   loss = 0.090375   (62.123 sec)
Epoch 20/23:   loss = 0.091075   (61.442 sec)
Epoch 21/23:   loss = 0.092651   (61.683 sec)
Epoch 22/23:   

## Evaluate model

In [10]:
print model.evaluate(train_set)
print model.evaluate(val_set)
print model.evaluate(test_set)

{'loss': 0.053702202}
{'loss': 0.091147743}
{'loss': 0.074106276}
